## Домашняя работа 10

ФИО: Филимонов Степан

Задание: обучите модель классификации букв для задачи расстановки ударения с помощью методов из библиотеки transformers. Датасет для обучения можно взять отсюда: https://github.com/Koziev/NLP_Datasets/blob/master/Stress/all_accents.zip

1. Напишите класс для Dataset/Dataloder и разбейте данные на случайные train / test сплиты в соотношении 50:50. (1 балл)
2. Попробуйте обучить одну или несколько из моделей: Bert, Albert, Deberta. Посчитайте метрику Accuracy на train и test. (1 балл). При преодолении порога в Accuracy на test 0.8: (+1 балл), 0.85: (+2 балла), 0.89: (+3 балла).
Пример конфигурации для deberta: https://huggingface.co/IlyaGusev/ru-word-stress-transformer/blob/main/config.json

In [19]:
import os
import sys
import time
import torch
import random
import string
import warnings
import datetime
import progressbar

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch import nn
from tqdm import tqdm
from torch import device, cuda
from transformers import BertTokenizer
from torch.nn import BCEWithLogitsLoss
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig

warnings.filterwarnings('ignore')

In [2]:
BATCH_SIZE = 32
DEVICE = device("cuda:0" if cuda.is_available() else "cpu")
print(DEVICE)

cuda:0


In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)

In [72]:
df = pd.read_csv('all_accents.tsv', delimiter='\t', names=['title', 'accent'])
df.sample(5)

,title,accent
1551,аболиционисский,аболицион^исский
849248,однопользовательских,одноп^ользовательских
1301354,растратишь,растр^атишь
1050413,подновляемый,подновл^яемый
53542,ассимилированными,ассимил^ированными


У нас будет задача классификации, где буква на которую падает ударение, есть истенный класс. Для этого в блоке `accent`
оставим только номер гласной, на которую падает уравнение, начиная с *1*. Если значение равно 0 - в слове нет ударения

In [73]:
df['accent'] = [ ( word.find('^') + 1 ) for word in  df['accent']]
NUM_CLASSES = (max(df["accent"]) - min(df["accent"]) + 1)
df['accent'] = [[ 0.0 if i != acc else 1.0 for i in range(NUM_CLASSES)] for acc in df['accent']]
print(f'Колличество данных: {df.shape[0]:,}')
print(f'Кол-во классов: {NUM_CLASSES}')

Колличество данных: 1,680,535
Кол-во классов: 56


In [74]:
MAX_LENGTH = max([len(tokenizer.encode(sent, add_special_tokens=True)) for sent in tqdm(df['title'])])
print('Максимальный размер предложения: ', MAX_LENGTH)

100%|██████████████████████████████| 1680535/1680535 [01:31<00:00, 18277.63it/s]

Максимальный размер предложения:  26


In [75]:
input_ids = []
attention_masks = []

for sent in tqdm(df['title']):
    encoded_dict = tokenizer.encode_plus(sent, add_special_tokens=True, max_length=MAX_LENGTH,
                                         pad_to_max_length=True, return_attention_mask=True,
                                         return_tensors='pt', truncation=True)

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(df['accent'])

100%|██████████████████████████████| 1680535/1680535 [02:20<00:00, 11929.80it/s]


In [76]:
dataset = TensorDataset(input_ids, attention_masks, labels)
train_dataset, val_dataset = train_test_split(dataset, train_size=0.5, random_state=42)
print(f'Обучающая выборка: {len(train_dataset):,}')
print(f'Валидация: {len(val_dataset):,}')

Обучающая выборка: 840,267
Валидация: 840,268


In [77]:
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=BATCH_SIZE)
validation_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=BATCH_SIZE)

In [139]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased", num_labels=NUM_CLASSES,
                                                      output_attentions=False, output_hidden_states=False,).to(DEVICE)
optimizer = AdamW(model.parameters(), lr=0.01)
model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [140]:
EPOCHS = 4
total_steps = len(train_dataloader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [145]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    _, lbls = torch.max(labels, 1)
    return torch.sum(preds == lbls).item() / lbls.shape[0]

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

#### Обучение

In [147]:
train_loss_epochs, train_accuracy_epochs = [], []
val_loss_epochs, val_accuracy_epochs = [], []

t = time.time()
for epoch_i in range(0, EPOCHS):
    print(
        f'======== Epoch {(epoch_i + 1)} / {EPOCHS} ========',
        f'\tTotal time: {format_time(time.time() - t)} s.',
        f'\n\tTraining...'
    )
    t0 = time.time()    
    
    total_train_loss, total_train_accuracy, total_eval_loss, total_eval_accuracy = 0.0, 0.0, 0.0, 0.0
    
    model.train()
    for batch in tqdm(train_dataloader):        
        batch = tuple(t.to(DEVICE) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        optimizer.zero_grad()
        output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels).logits
        loss = loss_foo(output, b_labels)
        loss.backward()
        optimizer.step()
        
        total_train_loss += loss.item()
        total_train_accuracy += accuracy(output, b_labels)
    
    train_loss_epochs.append(total_train_loss / len(train_dataloader))
    train_accuracy_epochs.append(total_train_accuracy / len(train_dataloader))

    print(
        f'\tTraining loss: {train_loss_epochs[-1]:.2f}.',
        f' Training accuracy: {train_accuracy_epochs[-1]:.2f}.'
        f' Time: {format_time(time.time() - t0)} s.',
        f'\n',
        f'\n\tRunning Validation...'
    )
    t0 = time.time()

    model.eval()
    for batch in tqdm(validation_dataloader):
        batch = tuple(t.to(DEVICE) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels).logits
        
        loss = loss_foo(output, b_labels)

        total_eval_loss += loss.item()
        total_eval_accuracy += accuracy(output, b_labels)

    val_accuracy_epochs.append(total_eval_accuracy / len(validation_dataloader))
    val_loss_epochs.append(total_eval_loss / len(validation_dataloader))
    
    print(
        f'\tVal loss: {val_loss_epochs[-1]:.2f}.',
        f' Val accuracy: {val_accuracy_epochs[-1]:.2f}.',
        f'Time: {format_time(time.time() - t0)} s.'
    )
    
    if val_accuracy_epochs[-1] >= 0.9:
        print('На обучающей и тестовой выборке достигли желаемого результата.\n',
              'Чтобы не израходовать ресурсы машины:\t break')
        break

======== Epoch 1 / 4 ======== 	Total time: 0:00:00 s. 
	Training...


100%|██████████████████████████████████████████████████████████████████████████████| 26259/26259 [34:59<00:00, 12.51it/s]


	Training loss: 0.07.  Training accuracy: 0.14. Time: 0:35:00 s. 
 
	Running Validation...


100%|██████████████████████████████████████████████████████████████████████████████| 26259/26259 [06:28<00:00, 67.57it/s]


	Val loss: 0.07.  Val accuracy: 0.15. Time: 0:06:29 s.
======== Epoch 2 / 4 ======== 	Total time: 0:41:28 s. 
	Training...


...

	Val loss: 0.08.  Val accuracy: 0.16. Time: 0:06:30 s.
======== Epoch 4 / 4 ======== 	Total time: 2:04:27 s. 
	Training...


100%|██████████████████████████████████████████████████████████████████████████████| 26259/26259 [34:57<00:00, 12.52it/s]


	Training loss: 0.07.  Training accuracy: 0.14. Time: 0:34:58 s. 
 
	Running Validation...


100%|██████████████████████████████████████████████████████████████████████████████| 26259/26259 [06:30<00:00, 67.28it/s]

	Val loss: 0.08.  Val accuracy: 0.03. Time: 0:06:30 s.


**Вывод:** 

#### До того как у меня появился доступ к gpu
~~Вычисления все полностью выходили бы примерно ~21 час. Исходя из опыта предыдущих домашних работ, если использовать весь массив,
для обучения и валидации модели, то для получения минимального loss достаточно пройти обучение, для экономии ресурса машины я решил до конца недообучать.~~

~~P.s. до конца семестра хочу получить доступ к машине с GPU, произведу пересчет данной задачи и дополню решение на портале~~

#### После того как у меня появился доступ к gpu

1. Не получилось достичь, accuracy 0.9. Похоже идея что номер ударения равен классу оказалась не удачной.

2. Похоже много классовой классификацией не стоит увлекатся, как-то очень размазано все вышло.

3. Без gpu я бы этого не понял, мне казалось изначально, что низкая accuracy получается из за того, что просто малую выборку выбираю